##**Section 1: Data Processing.**
When processing the data it is important to understand the context and the information that is relevant for the analysis.

#*1.1. Cleaning process with open refine.*
the first step while processing the data-set was removing the non-required dates and only considering April 2019, 2020, and 2021. At this point the raw data is obtained and the refining-data process can be started by dicarding information that is irrelevant for the practical work and carry out rectifications.

#*1.2. Setting the dates with the right format.*
after the removal of the unrequired months, the right formating of the dates was next, in order to carry out the corrections, the following steps were followed:

    1. Click on “CASH DATE” column
    2. edit cells, transform
    3. The following line of code was written: value.replace(“/”,”-”)

after following these steps we obtain the dates in a format thats easier to process: dd-mm-yyyy.


#*1.3. Changing the character-based format of the columns that had numerical information .*

In this stage the columns that required a numerical format reprocessing were (“NUMBER OF PERSONS INJURED”,”NUMBER OF PERSONS KILLED”,	“NUMBER OF PEDESTRIANS INJURED”,”NUMBER OF PEDESTRIANS KILLED”, “NUMBER OF CYCLIST INJURED”,”NUMBER OF CYCLIST KILLED”,”NUMBER OF MOTORIST INJURED”,	“NUMBER OF MOTORIST KILLED”)and the next steps were followed:

    1. Click on the respective tag of each column.
    2. Edit cells, Common transforms
    3. to number.

#*1.4. Retrieveing missing data.*

In this stage of the data processing adding the missing data from reverse geocoding was great alternative, the location is key information for the data analysis and since the column BOROUGH had several blank cells it was important to re-build the information that was missing in the dataset as much as possible. In order to do so the information available in other columns such as latitude and longitude could shed a ray of light for retrieving the missing boroughs since it contains implicitly that information.
The next steps were followed:

    1. Click on latitud column tag
    2. Edit column, add column by fetching URL
    3. The following code was written: https://nominatim.openstreetmap.org/reverse?lat="+value+"&lon="+cells["LONGITUDE"].value+"&format=json"
    4. A JSON file containing several information is retrieved where Borough that is the attribute of interest is also included


#*1.5. Reconstruction of Column Borough with the retrieved data from LATITUDE and LONGITUDE.*

At this point a good amount of information was recovered for Borough thanks to the available data in other columns. The next step in the data recovery process is to extract the information regarding the location, to be more precise "Borough", so it can be used to complete such Column. The following steps were followed:

    1. Click on column JSON
    2. Edit column, add column based on another column
    3. The following code was written: value.parse.Json().address.borough
    4. A new column with the newly retrieved data is created.
    5. Click on the new column tag, edit column, join column with column BOROUGH
    6. The blank spaces of the column BOROUGH are filled.

#*1.6. Retrieval of further location information with column ON STREET NAME.*

The first part for the reconstruction of the column BOROUGH is done, however there are still blank spaces to be filled and another column, which is ON STREET NAME can help us find the rest of the missing data, however the retrieving process is a little bit different thant the previous. In the following steps this new process is described:

    1. Click on column ON STREET NAME
    2. Edit column, add column by fetching URL
    3. The following code was written: 'https://nominatim.openstreetmap.org/search?format=json&q='+ escape(value,'url')
    4. A JSON file containing several information is retrieved where Borough that is the attribute of interest is also included.

#*1.7. Reconstruction of Column Borough with the retrieved data from ON STREET NAME.*

At this point we are in the final stage of data reconstruction for the column BOROUGH, the next thing that has to be done is the data extraction from the JSON file and the joining of the column BOROUGH with the new column that has been created out of the JSON, basically we need to repeat the previous steps that have been already described but this time with the new information.

    1. Click on column JSON
    2. Edit column, add column based on another column
    3. The following code was written: value.parse.Json().address.borough
    4. A new column with the newly retrieved data is created.
    5. Click on the new column tag, edit column, join column with column BOROUGH
    6. The blank spaces of the column BOROUGH are filled.


#*1.8. Removal of unnecesary information.*

Some columns in the data are not useful for the analysis, for instance there is no need of considering the "zip code", "latitude",  "longitude", "location", "cross street name", "off street name" and any other columns related to the location, as we already used them to retrieve the information for BOROUGH. On the other hand same applies for "collission ID" and as it does not offer valuable information for the objectives of this work.

the following code describes the step-by-step process for getting to the point that has been previously described.

    1. Click on each described column.
    2. Edit column, Remove this column.


#*1.9. Data Rectification and Correction.*

At this point we have all the necessary information to carry out the data visualization analysis, nontheless there are some Orthographic formating errors within the columns. Open refine has options such as facet, cluster and edit and replace which will help us in this final stage. The following steps were followed:

    1. Click on every column.
    2. Edit cells, cluster and edit.
    3. selection of all the equivalent words, set a unique identifier.
    4. Click on merge selected and Re-cluster, repeat until all the attributes are uniform and coherent.
    5. Another alternative, click on facet and check all the elements of the column, set a unique identifier word for equivalent attributes.

now we already have the cleaned dataset that will serve the objectives of this practical work.



In [ ]:
#Dataset import
import pandas as pd
from google.colab import files
import io

# csv file reading
upload = files.upload()
data= io.BytesIO(upload["cleanv1.csv"])

# Data-to-be-processed uploading step
refined_data = pd.read_csv(data)



Saving cleanv1.csv to cleanv1 (1).csv


##**Section 2: Project development.**

#*2.1. Have the accidents changed during stay-at-home period?*

In [ ]:

import altair as alt
import pandas as pd
import datetime

#The stay at home period is considered throughout the whole month of april 2020:
df=refined_data
df['CRASH DATE']=pd.to_datetime(df['CRASH DATE'])

# Grouping data by date 2019:
dat19=df[(df['CRASH DATE']<='30-04-19') & (df['CRASH DATE']>='01-04-19')]
dat19 = dat19['CRASH DATE'].value_counts().sort_index().rename_axis('2019').reset_index(name='Accidents')

# Grouping data by date 2020:
dat20=df[(refined_data['CRASH DATE']<='30-04-20') & (df['CRASH DATE']>='01-04-20')]
dat20 = dat20['CRASH DATE'].value_counts().sort_index().rename_axis('2020').reset_index(name='Accidents')

# Grouping data by date 2021:
dat21=refined_data[(df['CRASH DATE']<='30-04-21') & (df['CRASH DATE']>='01-04-21')]
dat21 = dat21['CRASH DATE'].value_counts().sort_index().rename_axis('2021').reset_index(name='Accidents')

# Construction of a Line chart containing the accidents per day and mean of accidents calculation for April 2020:
dat20['Weekday']= dat20['2020'].dt.day_name()

def row_function20 (row):
  row['2020']=str(row['2020'])[5:10]
  return row

month20=dat20.apply(row_function20, axis=1)
ap20=alt.Chart(month20).mark_line(size=2, point=True).encode(
    x='2020',
    y='Accidents',
    color=alt.value('#9e9e9e')
).properties(title='Number of accidents a day')

rule20 = alt.Chart(dat20).mark_rule(color='#000000',size=2).encode(
    y='mean(Accidents):Q'
)

#Construction of a Bar chart for the amount of accidents and the weekdays:
weekday20= alt.Chart(month20).mark_bar().encode(
    x='sum(Accidents)',
    y=alt.X('Weekday:O',title='',sort=["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"])
)
bars_weekday=weekday20.mark_bar().encode(
   color=alt.Color('Weekday:O',sort=["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"],
                   scale=alt.Scale(range=['#afafaf','#afafaf','#afafaf','#afafaf','#afafaf','#afafaf','#afafaf']),legend=None)
).properties(title='Number of accidents vs Weekday in 2020',width=300, height=200)
text_weekday = weekday20.mark_text(
    align='right',
    baseline='middle',
    color='black',
    dx=30
).encode(
    text='sum(Accidents)'
)


a_1=(ap20+rule20) | (bars_weekday+text_weekday)
a_1

alt.HConcatChart(...)

In [ ]:

# here we create a list of days for april.
#since april has the same ammount of days regardless the year we have chosen for instance 2020 for the creation of the list:
april_list=dat20.apply(row_function20, axis=1)
april_list.drop('Accidents', inplace=True, axis=1)
april_list=april_list['2020'].tolist()

#In this section a dataframe is created including the number of accidents per day for 2019, 2020 and 2021 respectively:
df_joint19=pd.DataFrame({"April":april_list,
                         'Year':['2019']*len(april_list),
                           'Number of Accidents':dat19['Accidents']})

df_joint20=pd.DataFrame({"April":april_list,
                         'Year':['2020']*len(april_list),
                           'Number of Accidents':dat20['Accidents']})

df_joint21=pd.DataFrame({"April":april_list,
                         'Year':['2021']*len(april_list),
                           'Number of Accidents':dat21['Accidents']})


df_all=df_joint19.append(df_joint20).append(df_joint21)

# Now using the prior iformation a graph containing information of each year is built:
#2019
joint19=alt.Chart(df_all).mark_line(size=2, point=True).encode(
    x='April',
    y='Number of Accidents',
    color=alt.Color('Year')
).properties(title='Accidents a day 2019-2020-2021')

a_2=joint19
a_2

alt.Chart(...)

In [ ]:
#With the following, the total of accidents for each year is calculated:
total19=dat19['Accidents'].sum()
total20=dat20['Accidents'].sum()
total21=dat21['Accidents'].sum()

#Data to be plotted, for 2019-2020-2021:
data_year=pd.DataFrame({"April":['2019','2020','2021'],
                           "Number of accidents":[total19,total20,total21]})

#Comparison of accidents among the desired years:
bar_total=alt.Chart(data_year).mark_bar(size=40).encode(
    alt.X('April', title=''),
    y='Number of accidents'
)
bars=bar_total.mark_bar().encode(
    color=alt.Color('April:N',scale=alt.Scale(range=['#afafaf','#210050','#afafaf']))
).properties(title='Total of accidents per year',width=150, height=300)
text = bar_total.mark_text(
    align='center',
    baseline='bottom',
    color='black'
).encode(
    text='Number of accidents:Q'
)
a_3=(bars + text)
a_3


alt.LayerChart(...)

##***2.2.Do accidents in 2021 look similar to 2019?***

In [ ]:
#Dataframe which contains the information between 2021 and 2019:
compare_year=pd.DataFrame({"April":['2019','2021'],
                           "Total of accidents":[float(total19),float(total21)]})

#conversion of dates into weekday names:
month19=dat19
month19['Weekday']= month19['2019'].dt.day_name()

month21=dat21
month21['Weekday']= month21['2021'].dt.day_name()

# A slope chart is built to analyse the behaviour of 2019 and 2021:
slope=alt.Chart(compare_year).mark_line().encode(
    x=alt.X('April:O', title=''),
    y='Total of accidents'
)
slope_compare=slope.mark_line(point=True, color='#afafaf').encode(
).properties(title='Decrease in total of accidents 2019 vs 2021', width=100, height=200)
text_slope = slope.mark_text(
    align='right',
    baseline='middle',
    color='black',
    dx=40
).encode(
    text='Total of accidents'
)

#Dot charts to display the number of accidents per weekday on april 2019 and 2021:
week_day_list=["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]

#2019
weekday19= alt.Chart(month19).mark_bar().encode(
    x=alt.X('sum(Accidents)',title='Accidents', scale=alt.Scale(domain=(1500,3500))),
    y=alt.Y('Weekday:O',title='',sort=week_day_list)
)
point_weekday19=weekday19.mark_bar().encode(
    color=alt.value('#afafaf')
).properties(title='Number of accidents vs Weekday in 2019', width=300, height=200)
text_weekday19 = weekday19.mark_text(
    align='right',
    baseline='middle',
    color='black',
    dx=40
).encode(
    text='sum(Accidents)'
)

#2021
weekday21= alt.Chart(month21).mark_bar().encode(
    x=alt.X('sum(Accidents)',title='Accidents',scale=alt.Scale(domain=(900,1700))),
    y=alt.Y('Weekday:O',title='',sort=week_day_list)
)
point_weekday21=weekday21.mark_bar().encode(
    color=alt.Color('Weekday:O',scale=alt.Scale(domain=week_day_list, range=['#afafaf','#afafaf','#afafaf','#afafaf','#afafaf','#afafaf','#afafaf']),legend=None)
).properties(title='Number of accidents vs Weekday in 2021',width=300, height=200)
text_weekday21 = weekday21.mark_text(
    align='right',
    baseline='middle',
    color='black',
    dx=38
).encode(
    text='sum(Accidents)'
)

b_1=(slope_compare+text_slope)|(point_weekday19+text_weekday19)|(point_weekday21+text_weekday21)
b_1

alt.HConcatChart(...)

In [ ]:
# As the required data is the difference between 2019 and 2021, the information is obtained with the following section of code:
def row_function19 (row):
  row['2019']=str(row['2019'])[5:10]
  return row
def row_function21 (row):
  row['2021']=str(row['2021'])[5:10]
  return row

merge21=dat21.apply(row_function21, axis=1)
merge19=dat19.apply(row_function19, axis=1)
datamerge=merge19.merge(merge21,left_on='2019',right_on='2021')
datamerge['Difference between years'] = datamerge['Accidents_x'] - datamerge['Accidents_y']

#Dataset to be plotted with the days of april and the difference between the desired years:
difference_joint=pd.DataFrame({'April':april_list,
                               'Year':['Difference']*len(april_list),
                           'Number of Accidents':datamerge['Difference between years']})


domain=['2019','2021','Difference']
df_all_difference=df_all[df_all['Year']!='2020'].append(difference_joint)

#Chart representation:
chart_all_difference=alt.Chart(df_all_difference).mark_line(size=2, point=True).encode(
    x='April',
    y='Number of Accidents',
    color=alt.Color('Year', scale=alt.Scale(domain= domain , range=['#4c78a8','#e45756','#3a3b3c']))
).properties(title='Accidents a day 2019 vs 2021')


#With the following we can have a unique chart, which is appropriate for the comparison among the graphs:
b_2=chart_all_difference
b_2

alt.Chart(...)

##***2.3.What about the number of victims, how has this behaved?***

This question will be analyside in three parts, first visualize how the number of victims have behaved during the dates of april 2019,2020 and 2021 respectively.

later on an analysis regarding the time when the accidents occur might be interesting to be considered and see what effects does it have over the number of victims

and finnally the relationship between the number of victims and the vehicle involved



In [ ]:
#getting the total of injured victims in 2019:
inj19=df[(refined_data['CRASH DATE']<='30-04-19') & (df['CRASH DATE']>='01-04-19')]
total_injured19=inj19['NUMBER OF PERSONS INJURED'].sum()
#getting the total of killed victims in 2019:
total_killed19=inj19['NUMBER OF PERSONS KILLED'].sum()

#Crash Victims on april 2019:
data_victims19=pd.DataFrame({"Victims":["Injured","Killed"],
                           "Number of Victims":[total_injured19,total_killed19]})

victims19=alt.Chart(data_victims19).mark_bar(size=40).encode(
    x='Victims',
    y='Number of Victims',
    color=alt.Color('Victims:N',
                   scale=alt.Scale(range=['#ffa833', '#c50909']))
).properties(title='April 2019',width=alt.Step(100), height=300)

#getting the total of injured victims in 2020:
inj20=refined_data[(df['CRASH DATE']<='30-04-20') & (df['CRASH DATE']>='01-04-20')]
total_injured20=inj20['NUMBER OF PERSONS INJURED'].sum()
#getting the total of killed victims in 2020:
total_killed20=inj20['NUMBER OF PERSONS KILLED'].sum()

#Crash Victims on april 2020:
data_victims20=pd.DataFrame({"Victims":["Injured","Killed"],
                           "Number of Victims":[total_injured20,total_killed20]})

#getting the total of injured victims in 2021:
inj21=refined_data[(df['CRASH DATE']<='30-04-21') & (df['CRASH DATE']>='01-04-21')]
total_injured21=inj21['NUMBER OF PERSONS INJURED'].sum()
#getting the total of killed victims in 2021:
total_killed21=inj21['NUMBER OF PERSONS KILLED'].sum()


#Data to be plotted for injured Victims:
data_vict_inj=pd.DataFrame({'Injured Victims':["Injured","Injured","Injured"],
                             'year':['2019','2020','2021'],
                           'Number of Victims':[total_injured19,total_injured20,total_injured21]})
#Data to be plotted for killed Victims:
data_vict_kill=pd.DataFrame({ 'Killed Victims':["Killed","Killed","Killed"],
                              'year':['2019','2020','2021'],
                             'Number of Victims':[total_killed19,total_killed20,total_killed21]})

#Joint chart of injured people:
injured=alt.Chart(data_vict_inj).mark_bar(color='#ec5d00').encode(
    x='year',
    y= 'Number of Victims',
    column='Injured Victims:N'
).properties(width=150, height=300)


#Joint chart of killed people:
killed=alt.Chart(data_vict_kill).mark_bar(color='#C70039').encode(
    x='year',
    y= 'Number of Victims',
    column='Killed Victims:N'
).properties(width=150, height=300)

c_1=injured|killed
c_1

alt.HConcatChart(...)

##***Time vs Number of victims:***

In [ ]:

#CRASH DATE Data filtering by the corresponding years:
df2_total=refined_data
df2_2019=refined_data[(df['CRASH DATE']<='30-04-19') & (df['CRASH DATE']>='01-04-19')]
df2_2020=refined_data[(df['CRASH DATE']<='30-04-20') & (df['CRASH DATE']>='01-04-20')]
df2_2021=refined_data[(df['CRASH DATE']<='30-04-21') & (df['CRASH DATE']>='01-04-21')]

#2019
df2_2019=pd.to_datetime(df2_2019['CRASH TIME']).dt.strftime('%H:00-%H:59')
out_2019=df2_2019.groupby(df2_2019, sort=True).size()
df2_2019= out_2019.rename_axis('Time of the day').reset_index(name='Accidents')
df2_2019['Year']='2019'

#2020
df2_2020=pd.to_datetime(df2_2020['CRASH TIME']).dt.strftime('%H:00-%H:59')
out_2020=df2_2020.groupby(df2_2020, sort=True).size()
df2_2020= out_2020.rename_axis('Time of the day').reset_index(name='Accidents')
df2_2020['Year']='2020'

#2021
df2_2021=pd.to_datetime(df2_2021['CRASH TIME']).dt.strftime('%H:00-%H:59')
out_2021=df2_2021.groupby(df2_2021, sort=True).size()
df2_2021= out_2021.rename_axis('Time of the day').reset_index(name='Accidents')
df2_2021['Year']='2021'

#Total
df2_total=pd.to_datetime(df2_total['CRASH TIME']).dt.strftime('%H:00-%H:59')
out_total=df2_total.groupby(df2_total, sort=True).size()
df2_total= out_total.rename_axis('Time of the day').reset_index(name='Accidents')
df2_total['Year']='Total'

df2_all_hourly= df2_2019.append(df2_2020).append(df2_2021).append(df2_total)
#Charts representing the information for each year:
#all charts:
total19_chart=alt.Chart(df2_all_hourly).mark_line(size=2, point=True).encode(
    x='Time of the day',
    y='Accidents',
    color=alt.Color('Year', scale=alt.Scale(range=['#4c78a8','#f58518','#e45756','#3a3b3c']))
).properties(title='Number of accidents vs time', width=500)


#Joint chart of Number of accidents vs time:
c_2=total19_chart
c_2

alt.Chart(...)

##***Kind of victim and outcome:***

In [ ]:
#(PEDESTRIANS)
#getting the total of injured victims:
pedestrians_injured=refined_data['NUMBER OF PEDESTRIANS INJURED'].sum()
#getting the total of killed victims:
pedestrians_killed=refined_data['NUMBER OF PEDESTRIANS KILLED'].sum()

#Pedestrians Data:
data_pedestrians=pd.DataFrame({"Victims":["Injured","Killed"],
                           "Number of Victims":[pedestrians_injured,pedestrians_killed]})

#(CYCLISTS)
#getting the total of injured victims:
cyclist_injured=refined_data['NUMBER OF CYCLIST INJURED'].sum()
#getting the total of killed victims:
cyclist_killed=refined_data['NUMBER OF CYCLIST KILLED'].sum()

#Cyclist Data:
data_cyclist=pd.DataFrame({"Victims":["Injured","Killed"],
                           "Number of Victims":[cyclist_injured,cyclist_killed]})

#(MOTORISTS)
#getting the total of injured victims:
motorist_injured=refined_data['NUMBER OF MOTORIST INJURED'].sum()
#getting the total of killed victims:
motorist_killed=refined_data['NUMBER OF MOTORIST KILLED'].sum()

#Motorists Data:
data_motorist=pd.DataFrame({"Victims":["Injured","Killed"],
                           "Number of Victims":[motorist_injured,motorist_killed]})

#Data to be plotted containing the category of all the injured victims:
df_type_inj=pd.DataFrame({'Victims':["Injured","Injured","Injured"],
                             'type of victim':['Motorists','Pedestrian','Cyclist'],
                           'Number of Victims':[motorist_injured,pedestrians_injured,cyclist_injured]})

#Data to be plotted containing the category of all the killed victims:
df_type_kill=pd.DataFrame({'Victims':["Killed","Killed","Killed"],
                             'type of victim':['Motorists','Pedestrian','Cyclist'],
                           'Number of Victims':[motorist_killed,pedestrians_killed,cyclist_killed]})


#Injured victims chart:
type_inj=alt.Chart(df_type_inj).mark_bar(size=40).encode(
    x=alt.X('type of victim:N', title='',sort='-y'),
    y= 'Number of Victims:Q'
)

bar_type_inj=type_inj.mark_bar(size=40).encode(
     color=alt.Color('type of victim:N', sort='-y',
                   scale=alt.Scale(range=['#210050','#9e9e9e','#9e9e9e']), legend=None)
).properties(title='Total of injured Victims by type',width=150, height=300)
text_type_inj = type_inj.mark_text(
    align='center',
    baseline='bottom',
    color='black'
).encode(
    text='Number of Victims:Q'
)

#Killed victims chart:
type_kill=alt.Chart(df_type_kill).mark_bar().encode(
    x=alt.X('type of victim:N',  title='',sort='-y'),
    y= 'Number of Victims'
)
bar_type_kill=type_kill.mark_bar(size=40).encode(
     color=alt.Color('type of victim:N', sort='-y',
                   scale=alt.Scale(range=['#210050','#9e9e9e','#9e9e9e']),legend=None)
).properties(title='Total of killed Victims by type',width=150, height=300)
text_type_kill = type_kill.mark_text(
    align='center',
    baseline='bottom',
    color='black'
).encode(
    text='Number of Victims:Q'
)

c_3=(bar_type_inj+text_type_inj)|(bar_type_kill+text_type_kill)
c_3

alt.HConcatChart(...)

##***2.4.Are there any areas with larger number of accidents?***

In [ ]:
#Here the total of accident per area is obtained:
dat_areas=refined_data['BOROUGH'].value_counts().rename_axis('Areas').reset_index(name='Number of Accidents')

#Chart representing the total of accidents vs Location:
areas=alt.Chart(dat_areas).mark_point().encode(
    x='Number of Accidents',
    y=alt.Y('Areas:N', title='', sort='-x'),
)
dot_areas=areas.mark_point(filled=True, size=100, opacity=1).encode(
    color=alt.Color('Areas:N',sort='-x',
                   scale=alt.Scale(range=['#210050','#9e9e9e','#9e9e9e','#9e9e9e','#9e9e9e','#9e9e9e']),legend=None)
).properties(title='Total of Accidents vs Location',width=300, height=200)
text_areas = areas.mark_text(
    align='right',
    baseline='middle',
    color='black',
    dx=38
).encode(
    text='Number of Accidents:Q'
)

d_1=(dot_areas+text_areas)
d_1

alt.LayerChart(...)

In [ ]:
#Actual population of New york:

#Bronx	Borough =	1472654
#Brooklyn Borough	= 2736074
#Manhattan Borough =	1694251
#Queens	Borough	= 2405464
#Staten Island Borough = 495747


#we save the aforementioned information in an array-list and create another containing the areas:
population=[2736074,2405464,1694251,1472654,495747]
Boroughs=['BROOKLYN','QUEENS','MANHATTAN','BRONX','STATEN ISLAND']

#In the next section of code the mathematical calculation is carried out in order to get the number of citizens per accident occurred:
acumula=[]
for i in range(0,len(dat_areas['Number of Accidents'])-1):
  valor=population[i]/dat_areas['Number of Accidents'][i]
  acumula.append(valor)
acumula= [round(x) for x in acumula]

#Data-set to be plotted:
pop_borugh=pd.DataFrame({'Areas':Boroughs,
                           'Accidents per citizen':acumula})


#Accidents per population chart:
area_pop=alt.Chart(pop_borugh).mark_point().encode(
    x=alt.X('Accidents per citizen', scale=alt.Scale(domain=(250,600))),
    y=alt.Y('Areas:N', title='', sort='-x')
)
dot_areas_pop=area_pop.mark_point(filled=True, size=100, opacity=1).encode(
    color=alt.Color('Areas:N',sort='-x',
                   scale=alt.Scale(range=['#210050','#9e9e9e','#9e9e9e','#9e9e9e','#9e9e9e','#9e9e9e']),legend=None)
).properties(title='Citizens per accident vs Location',width=300, height=200)
text_pop = area_pop.mark_text(
    align='right',
    baseline='middle',
    color='black',
    dx=35
).encode(
    text='Accidents per citizen:Q'
)

d_2=(dot_areas_pop+text_pop)
d_2

alt.LayerChart(...)

##***2.5.What are the main reasons of accidents?***



In [ ]:
df4=refined_data

#All the data that contains information about the reason of accidentality is gathered with the following section of code:
out=df4['CONTRIBUTING FACTOR VEHICLE 1'].append(df4['CONTRIBUTING FACTOR VEHICLE 2']).reset_index(drop=True)
out=out.append(df4['CONTRIBUTING FACTOR VEHICLE 3']).reset_index(drop=True)
out=out.append(df4['CONTRIBUTING FACTOR VEHICLE 4']).reset_index(drop=True)
out=out.append(df4['CONTRIBUTING FACTOR VEHICLE 5']).reset_index(drop=True)

#Then the ammount of accidents per contributing factor is obtained:
dat_reasons=out.value_counts().rename_axis('Reasons').reset_index(name='Accidents')

#and the top ten reasons are plotted:
dat_reasons= dat_reasons.iloc[1:].head(10)



chart_reasons=alt.Chart(dat_reasons).mark_bar().encode(
    x=alt.X('Accidents',scale=alt.Scale(domain=(0,11000))),
    y=alt.Y('Reasons:N', title='', sort='-x')
)
bar_reasons=chart_reasons.mark_bar().encode(
    color=alt.Color('Reasons:N', sort='-x',
                   scale=alt.Scale(range=['#210050','#9e9e9e','#9e9e9e','#9e9e9e','#9e9e9e','#9e9e9e','#9e9e9e','#9e9e9e','#9e9e9e','#9e9e9e']),legend=None)
).properties(title='Main reasons of accidents',width=300, height=200)
text_reasons = chart_reasons.mark_text(
    align='right',
    baseline='middle',
    color='black',
    dx=35
).encode(
    text='Accidents'
)

e_1=(bar_reasons+text_reasons)
e_1

alt.LayerChart(...)

###***Relationship between number accidents and vehicle involved:***###

In [ ]:
df5=refined_data

#All the data that contains information about the vehicle involved in the accident is gathered, with the following section of code:
out2=df5['VEHICLE TYPE CODE 1'].append(df5['VEHICLE TYPE CODE 2']).reset_index(drop=True)
out2=out2.append(df5['VEHICLE TYPE CODE 3']).reset_index(drop=True)
out2=out2.append(df5['VEHICLE TYPE CODE 4']).reset_index(drop=True)
out2=out2.append(df5['VEHICLE TYPE CODE 5']).reset_index(drop=True)

#Then the ammount of accidents per vehicle involved is obtained:
dat_vehicle=out2.value_counts().rename_axis('Vehicle').reset_index(name='Accidents')

#and the top ten is plotted:
dat_vehicle= dat_vehicle.iloc[1:].head(10)

chart_vehicle=alt.Chart(dat_vehicle).mark_bar().encode(
    x=alt.X('Accidents',scale=alt.Scale(domain=(0,28000))),
    y=alt.Y('Vehicle:N',title='', sort='-x')
)
bar_vehicle=chart_vehicle.mark_bar().encode(
    color=alt.Color('Vehicle:N',sort='-x',
                   scale=alt.Scale(range=['#210050','#9e9e9e','#9e9e9e','#9e9e9e','#9e9e9e','#9e9e9e','#9e9e9e','#9e9e9e','#9e9e9e','#9e9e9e']), legend=None)
).properties(title='Type of Vehicle involved vs Number of accidents',width=300, height=200)
text_vehicle = chart_vehicle.mark_text(
    align='right',
    baseline='middle',
    color='black',
    dx=40
).encode(
    text='Accidents'
)

e_2=(bar_vehicle+text_vehicle)
e_2

alt.LayerChart(...)

In [ ]:
row1=(a_1 | a_2 |a_3 ).resolve_scale(
    color='independent'
)
row2=(b_1|b_2).resolve_scale(
    color='independent'
)
row3=(c_1|c_2 |c_3).resolve_scale(
    color='independent'
)
row4=(d_1|d_2 |e_1|e_2).resolve_scale(
    color='independent'
)
dashboard=(row1&row2&row3&row4).resolve_scale(
    color='independent'
)
dashboard

alt.VConcatChart(...)

##***ANSWERS TO QUESTIONS:***



##***2.1. Have the accidents changed during stay-at-home period?***

In order to understand the behavior of accidents during the lock down period, we have to consider the whole April 2020 data. As a first approach we made 2 different charts, a line chart time series with all the daily information of April 2020 and an horizontal bar chart that included information about the weekdays. We can see that in the chart "number of accidents a day" the highest rates of accidents occur in both the beginning and the end of the month for example we can highlight the 3th, 24th, 28th and 30th of April where we can spot peaks in accidents and all those values are over the mean of accidents.

On the other hand with the bar chart "number of accidents vs weekday in 2020" we can see that the total of accidents increase gradually until reaching the day with highest accidents which is Wednesday with 703 accidents and curiously after that day it starts to decrease at the same rate, probably because it was gettign closer to the weekend.  

Another factor that was reviewed for this question was the number of accidents a day throughout the whole month of April for 2019,2020 and 2021 so we could see how the lock down period in 2020 behaved in contrast to 2019 and 2021. Now the line graph "accidents a day 2019-2020-2021" contains information of each year where we can easily see that effectively 2020 is the year with the lowest amount of accidents, ranging from nearly 100 and 200 accidents a day. On the other hand 2021 follows with a lowest near to 200 and a highest close to 400, a value that is very close to the lowest value of 2019, being this year the highest in terms of accidents reaching values near to 700.

finally the bar chart "total of accidents per year" gives us more clarity on the previous described behavior but in a greater perspective where the total of accidents a year for the month of April are considered, here we can see that 2020 is the year with the lowest number of accidents with a value of 4127 accidents, then follows 2021 with a total number of accidents of 8749 and finally the highest which is 2019 with 16829 accidents something to remark is that this value doubles 2021 and also is 4 times greater thant the value of 2020.  

##***2.2.Do accidents in 2021 look similar to 2019?***



In this section we have several visualizations that can help us find a good answer to this question. To begin with the analysis, we  have a slope chart
"decrease in total of accidents 2019 vs 2021" that instantly tells us the decreasing-like behavior of the number of accidents, that we already witnessed with a first approach in the previous question. On the other hand we have 2 bar charts that let us analyze how the number of accidents have behave in terms of weekdays for both years,in "number of accidents vs weekday 2019" we can see that in 2019, Monday and Tuesday are the days with the highest number of accidents with 2940 and 2907 accidents respectively then as the days go by, this pattern begins to decrease reaching a low on Sunday at 1737 accidents. Now considering "number of accidents vs weekday in 2021" this year shows a rising behavior starting at 1005 on Sunday and reaching the highest number on Friday at 1550 accidents, then after that, drops dramatically to 1140 accidents on Saturday, this behavior let us now that on weekends looks like there are less accidents than weekdays in the case of April 2021.

In order to see how 2019 and 2021 behave on a daily basis we have made a time-series line chart where the accidents per day and the difference between those 2 years is pictured. In "accidents a day 2019 vs 2021" we can observe  that 2019 is notoriously greater than 2021 and has a peak on April 5th however there is one particular day that catches the attention. As we can spot on April 28th the difference between the number of accidents in 2019 and 2021 is very small and it is the only day where 2021 was closer to 2019, but it is important to highlight that 2021 and 2019 show the same behavior since they both have almost the same peaks and the same lows during April, a possible explanation to this might be the celebration of a holiday such as Worker's Memorial Day.


##***2.3.What about the number of victims, how has this behaved?***

The number of victims is incredibly high and a lot of information is obtained from the next charts. First we have 2 bar charts "Injured victims" and  "killed victims" charts where we can analyze what happens with the behaviour of the victims for each year. In the "Injured victims"and "killed victims" chart respectively we can see that in 2019 there are 4811 injured victims and 20 killed, then we observe that for 2020 there are 1312 injured and 14 killed , as for 2021 we see a rise again showing 4094 injured and 27 killed. Again 2020 is the year with less victims both injured and killed, however 2021 has the highest number of killed victims outnumbering 2019 that has had the highest numbers up until this point.

In the next section we have chart that compares the number of accidents among the years througout the time,here we can have a better approach on how the accidents  have behaved under this condition.  As we can see in the "number of accidents vs time" chart, in the mornings the accidents begin to increase just right after 5 am which is the time where most of New Yorkers begin their busy day, a first peak is observed at 8 am then it continues to rise reaching another at 2pm, a final peak and the highest is spotted at 4 pm where the propbable reason for this time to be the highest is that most citizens are leaving their work locations for home so the risk of having an accident grows, soon after that time interval it begins to lower down displaying a bell-like shape. The years 2021 and 2020 show a rather steady behavior nonetheless it is possible to see that they follow the same pattern than the other years.

Now depending on the victim we can see different numbers, for instance in the bar chart "total of injured victims by type" we see that in regard to injured victims the motorist are the most affected with a value of 7739, then follows the pedestrians with 1435 and finally the cyclist with an amount of 875 accidents. On the other hand by seeing the chart "total of killed victims by type" the killed victims have the motorists leading the group with 31 deaths, then the pedestrians with 24 deaths and last the cyclist with just 4 deaths.


##***2.4.Are there any areas with larger number of accidents?***

Finally we are going to analyze the areas with most accidents where we used dot charts that make it easier to see the values with small changes just like in this case, in the dot chart "total of accidents vs location" we can see that Brooklyn is leading with 9007 accidents then close to it is queens with 8551, Manhattan and Bronx very close to each other with 5267 and 5213 respectively, last we find Staten island with 608 accidents.

Then as a second analysis we made another dot chart "citizens per accident vs location" where we wanted to analyze the number of population when an accident happen depending on the location. Surprisingly we see that Staten Island is leading this time but as the safest suburb to live in as for every 468 citizen one accident happen, on the other hand Bronx and queens are the most dangerous as for every 282 and 281 citizens respectively occur 1 accident.


##***2.5.What are the main reasons of accidents?***

For the main reasons of accident we have made a top 10 where we can see how the values change depending on the reasons of why the accident happened based on the bar chart "main reasons of accident" we have the following :

1.   Driver Inattention/Distraction       9283
2.            Following Too Closely       2744
3.   Passing or Lane Usage Improper       2442
4.    Failure to Yield Right-of-Way       2261
5.                  Other Vehicular       1454
6.              Passing Too Closely       1349
7.                 Backing Unsafely       1285
8.                     Unsafe Speed        928
9.                  Traffic Control        794
10.              Turning Improperly        691

so the reason that causes more accidents is Driver Inattention/Distraction with 9283 accidents.

then in regard to the type of vehicle that causes more accident we have made a top 10 where we can see how the values change depending on the vehicle involved in the accidents based on the bar chart "type of vehicle involved vs number of accidents" we have the following:

1.                                Sedan      24923
2.  Station Wagon/Sport Utility Vehicle      19827
3.                                Truck       4026
4.                                  Cab       1976
5.                                 Bike       1288
6.                                  Bus        913
7.                                  Van        396
8.                            Motorbike        389
9.                            Ambulance        252
10.                              Scooter        164

where the vehicle that have been involved in more accidents is the type sedan with 24923 accidents.